In [5]:
import pandas as pd
import numpy as np
import os
############################ data clean
def basic():
    #read all docs
    A=pd.read_csv(r".\input_alabaster\alabaster_20160101_tradelist.csv")
    B=pd.read_csv(r".\input_bernstein\bernstein_TradeInputList_160101.csv")
    C=pd.read_csv(r".\input_comwealth\trades_comwealth_jan2016.csv")
    M=pd.read_csv(r".\mapping\MappingTable.csv")

    #define col to drop
    A_to_drop=["PriceEUR","MktAdjFactor","AllocationID","OrderQty","FilledQty","FinancingSpread"]
    B_to_drop=["MTMValue","Currency","Financing","SubAccount"]

    #drop the cols
    A=A.drop(columns=A_to_drop)
    B=B.drop(columns=B_to_drop)

    #create security map dics
    dictA=dict(zip(M.BB_ID,M.Security))## BBID-->Security
    dictB=dict(zip(M.GENERIC,M.Security))## GenericTicker	-->Security
    dictC=dict(zip(M.RIC,M.Security))## Ric-->Security

    #replace based on dics
    A=A.replace({"BB_ID": dictA})
    B=B.replace({"GenericTicker": dictB})
    C=C.replace({"RIC$Code": dictC})

    #rename the cols
    A.rename(columns={'BB_ID':'Security',"PriceUSD":"Price_A",
                      "Qty":"Quantity_A", "Beta":"Beta_A","Date":"Date_A"
                     }, inplace=True)
    B.rename(columns={'GenericTicker':'Security',"Price":"Price_B",
                      "Quantity":"Quantity_B", "Beta":"Beta_B","Date":"Date_B"
                     }, inplace=True)
    C.rename(columns={'RIC$Code':'Security',"Executed$Price":"Price_C",
                      "Executed$Quantity":"Quantity_C", "Beta$Hundreds":"Beta_C","Trade$Date":"Date_C"
                     }, inplace=True)

    #format date
    A.Date_A=pd.to_datetime(A.Date_A)

    ########################## data process
    #operation
    op=pd.merge(pd.merge(A,B,on="Security"),C,on="Security").drop(columns=["Date_B","Date_C"]).rename(columns={'Date_A':'Date'})
    result=op[(op["Price_A"]!=op["Price_B"])|
        (op["Price_C"]!=op["Price_B"])|
       (op["Price_A"]!=op["Price_B"])]

    #trade
    op["AveragePrice"]=(op["Price_A"]+op["Price_B"]+op["Price_C"])/3
    op["TotalQuantity"]=(op["Quantity_A"]+op["Quantity_B"]+op["Quantity_C"])/3
    op["MaxBeta"]=op[["Beta_A","Beta_B","Beta_C"]].max(axis=1)

    #########################data ouput
    outdir = r'.//output-operations'
    outdir1 = r'.//output-trading'
    outname="output.csv"
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if not os.path.exists(outdir1):
        os.mkdir(outdir1)

    fullname = os.path.join(outdir, outname)   
    fullname1 = os.path.join(outdir1, outname)    

    result.to_csv(fullname)
    op.to_csv(fullname1)
    
    print("done")

def main():
    basic()

if __name__=="__main__":
    main()

done
